# **Proyecto Final Herramientas de BigData**

**Maestria en Analitica Aplicada**

**Universidad de La Sabana, Facultad de Ingeniería**


Presentado por: 
-   Julián Andrés Santos Méndez
-   Juan Felipe Sepulveda Mantilla
-   Luz Edilsa Ortiz Lopez
-   Julian Obando Scarpetta

Archivo: Transformacion de datos crudos para analitica

In [125]:
import pandas as pd
import os
import pandas as pd
from sqlalchemy import create_engine
import dotenv


## Conexión a la Base de Datos Crudos

In [126]:
dotenv.load_dotenv("vars.env", override=True)

db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

conn = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}', echo=False)

## Transformacion Variables de Generación - Sistema

### Generación del Sistema
-   Metrica: Generacion Sistema
-   Unidad: kWh	
-   Descripción: Generacion neta de cada una de las plantas Nacionales en sus puntos de frontera.

In [127]:
sql_query = """SELECT * FROM generacion_sistema"""
df_generacion_sistema = pd.read_sql_query(sql_query, con=conn)
df_generacion_sistema['Date'] = pd.to_datetime(df_generacion_sistema['Date'])
df_generacion_sistema

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23,5296750.71,4919467.58,4638776.22,4442074.76,4279026.64,3827254.47,3832573.95,...,4391372.59,4365640.95,4605721.12,6021430.06,6527391.35,6261393.45,5776398.57,5211439.50,4714078.01,2010-01-01
1,Sistema,Sistema,4443442.83,4238022.31,4153685.87,4103574.79,4153986.60,4252501.43,4185993.85,4479587.60,...,5397357.88,5317214.52,5482854.99,6895377.37,7356071.75,7109606.16,6611477.08,5931707.62,5211930.81,2010-01-02
2,Sistema,Sistema,4771417.74,4520041.29,4364500.03,4210941.04,4205070.55,4312551.86,4156670.92,4331637.24,...,5015866.10,4981069.04,5169045.72,6637669.36,7126930.30,6924827.71,6432570.74,5752050.08,5135978.48,2010-01-03
3,Sistema,Sistema,4705250.02,4493314.06,4399068.67,4395569.56,4475996.61,4777750.57,4911096.21,5322214.49,...,6618343.03,6458778.34,6529860.50,7819390.86,8232402.15,7884258.05,7197982.57,6288573.58,5535400.79,2010-01-04
4,Sistema,Sistema,5171964.92,4939629.25,4840107.60,4774763.87,4949684.01,5195571.83,5362721.47,5742326.06,...,6778398.35,6715045.20,6745497.24,8023899.14,8336888.23,7929022.67,7243496.48,6378988.75,5737144.04,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80,8198137.14,7966493.50,7848538.15,7993954.30,8373795.23,8533971.50,8989609.97,...,10224371.17,10138968.61,9944771.52,10267157.48,10388113.98,10186516.32,9774795.94,9269233.15,8783850.22,2024-04-26
5230,Sistema,Sistema,8416268.81,8139684.03,7934365.14,7801216.87,7802711.66,7866953.71,8060693.66,8611512.50,...,9613448.98,9515583.90,9453804.19,10022279.38,10070282.41,9806988.02,9391935.06,8934686.71,8515651.18,2024-04-27
5231,Sistema,Sistema,8153358.39,7823646.09,7495560.68,7290789.06,7186795.65,7058942.05,6899473.59,7167562.93,...,8373980.81,8391670.64,8366776.05,9037799.05,9296771.60,9194946.81,8963686.66,8545982.69,8101750.18,2024-04-28
5232,Sistema,Sistema,7755006.39,7559477.81,7424995.07,7358043.41,7541190.63,7993483.44,8218854.74,8737461.99,...,10365625.70,10314129.94,10040895.25,10521997.09,10740452.13,10521840.29,10125689.57,9473347.41,8842359.25,2024-04-29


In [128]:
df_generacion_sistema = df_generacion_sistema.set_index('Date')
df_generacion_sistema

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour15,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,Sistema,Sistema,5609982.23,5296750.71,4919467.58,4638776.22,4442074.76,4279026.64,3827254.47,3832573.95,...,4457991.95,4391372.59,4365640.95,4605721.12,6021430.06,6527391.35,6261393.45,5776398.57,5211439.50,4714078.01
2010-01-02,Sistema,Sistema,4443442.83,4238022.31,4153685.87,4103574.79,4153986.60,4252501.43,4185993.85,4479587.60,...,5473192.84,5397357.88,5317214.52,5482854.99,6895377.37,7356071.75,7109606.16,6611477.08,5931707.62,5211930.81
2010-01-03,Sistema,Sistema,4771417.74,4520041.29,4364500.03,4210941.04,4205070.55,4312551.86,4156670.92,4331637.24,...,5089743.12,5015866.10,4981069.04,5169045.72,6637669.36,7126930.30,6924827.71,6432570.74,5752050.08,5135978.48
2010-01-04,Sistema,Sistema,4705250.02,4493314.06,4399068.67,4395569.56,4475996.61,4777750.57,4911096.21,5322214.49,...,6702923.38,6618343.03,6458778.34,6529860.50,7819390.86,8232402.15,7884258.05,7197982.57,6288573.58,5535400.79
2010-01-05,Sistema,Sistema,5171964.92,4939629.25,4840107.60,4774763.87,4949684.01,5195571.83,5362721.47,5742326.06,...,6854886.19,6778398.35,6715045.20,6745497.24,8023899.14,8336888.23,7929022.67,7243496.48,6378988.75,5737144.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-26,Sistema,Sistema,8518210.80,8198137.14,7966493.50,7848538.15,7993954.30,8373795.23,8533971.50,8989609.97,...,10199680.49,10224371.17,10138968.61,9944771.52,10267157.48,10388113.98,10186516.32,9774795.94,9269233.15,8783850.22
2024-04-27,Sistema,Sistema,8416268.81,8139684.03,7934365.14,7801216.87,7802711.66,7866953.71,8060693.66,8611512.50,...,9669628.56,9613448.98,9515583.90,9453804.19,10022279.38,10070282.41,9806988.02,9391935.06,8934686.71,8515651.18
2024-04-28,Sistema,Sistema,8153358.39,7823646.09,7495560.68,7290789.06,7186795.65,7058942.05,6899473.59,7167562.93,...,8364289.97,8373980.81,8391670.64,8366776.05,9037799.05,9296771.60,9194946.81,8963686.66,8545982.69,8101750.18


In [129]:
df_generacion_sistema['generacion_sistema_daily'] = df_generacion_sistema.loc[:, 'Values_Hour01':'Values_Hour24'].sum(axis=1)
df_generacion_sistema

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,generacion_sistema_daily
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,Sistema,Sistema,5609982.23,5296750.71,4919467.58,4638776.22,4442074.76,4279026.64,3827254.47,3832573.95,...,4391372.59,4365640.95,4605721.12,6021430.06,6527391.35,6261393.45,5776398.57,5211439.50,4714078.01,1.154191e+08
2010-01-02,Sistema,Sistema,4443442.83,4238022.31,4153685.87,4103574.79,4153986.60,4252501.43,4185993.85,4479587.60,...,5397357.88,5317214.52,5482854.99,6895377.37,7356071.75,7109606.16,6611477.08,5931707.62,5211930.81,1.276812e+08
2010-01-03,Sistema,Sistema,4771417.74,4520041.29,4364500.03,4210941.04,4205070.55,4312551.86,4156670.92,4331637.24,...,5015866.10,4981069.04,5169045.72,6637669.36,7126930.30,6924827.71,6432570.74,5752050.08,5135978.48,1.236378e+08
2010-01-04,Sistema,Sistema,4705250.02,4493314.06,4399068.67,4395569.56,4475996.61,4777750.57,4911096.21,5322214.49,...,6618343.03,6458778.34,6529860.50,7819390.86,8232402.15,7884258.05,7197982.57,6288573.58,5535400.79,1.459575e+08
2010-01-05,Sistema,Sistema,5171964.92,4939629.25,4840107.60,4774763.87,4949684.01,5195571.83,5362721.47,5742326.06,...,6778398.35,6715045.20,6745497.24,8023899.14,8336888.23,7929022.67,7243496.48,6378988.75,5737144.04,1.521480e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-26,Sistema,Sistema,8518210.80,8198137.14,7966493.50,7848538.15,7993954.30,8373795.23,8533971.50,8989609.97,...,10224371.17,10138968.61,9944771.52,10267157.48,10388113.98,10186516.32,9774795.94,9269233.15,8783850.22,2.246048e+08
2024-04-27,Sistema,Sistema,8416268.81,8139684.03,7934365.14,7801216.87,7802711.66,7866953.71,8060693.66,8611512.50,...,9613448.98,9515583.90,9453804.19,10022279.38,10070282.41,9806988.02,9391935.06,8934686.71,8515651.18,2.171655e+08
2024-04-28,Sistema,Sistema,8153358.39,7823646.09,7495560.68,7290789.06,7186795.65,7058942.05,6899473.59,7167562.93,...,8373980.81,8391670.64,8366776.05,9037799.05,9296771.60,9194946.81,8963686.66,8545982.69,8101750.18,1.941888e+08


In [130]:
df_generacion_sistema_daily = df_generacion_sistema[['generacion_sistema_daily']]
df_generacion_sistema_daily

,generacion_sistema_daily
Date,
2010-01-01,1.154191e+08
2010-01-02,1.276812e+08
2010-01-03,1.236378e+08
2010-01-04,1.459575e+08
2010-01-05,1.521480e+08
...,...
2024-04-26,2.246048e+08
2024-04-27,2.171655e+08
2024-04-28,1.941888e+08


### Generación del Sistema Ideal
-   Metrica: Generacion Sistema Ideal
-   Unidad: kWh	
-   Descripción: Despacho de generacion que resulta de considerar una red de transporte inexistente

In [131]:
sql_query = """SELECT * FROM generacion_ideal_sistema"""
df_generacion_ideal_sistema = pd.read_sql_query(sql_query, con=conn)
df_generacion_ideal_sistema['Date'] = pd.to_datetime(df_generacion_ideal_sistema['Date'])
df_generacion_ideal_sistema

,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Sistema,Sistema,5609982.23,5299387.09,4922337.76,4640952.55,4443431.20,4280072.11,3828774.82,3832979.13,...,4392140.29,4366427.63,4606016.29,6021430.06,6527391.35,6263172.04,5779281.34,5212138.25,4714247.65,2010-01-01
1,Sistema,Sistema,4445348.51,4238116.21,4154887.32,4103709.79,4153986.60,4252501.43,4186498.45,4480615.46,...,5397357.88,5318190.02,5484348.49,6895377.37,7356107.90,7110891.37,6612818.22,5934135.44,5214100.60,2010-01-02
2,Sistema,Sistema,4772268.38,4520104.69,4365723.51,4211679.57,4205364.41,4312551.86,4156670.92,4333251.97,...,5016125.95,4981354.24,5169667.20,6637669.36,7126930.30,6927793.22,6433970.64,5753971.20,5137059.49,2010-01-03
3,Sistema,Sistema,4705845.03,4493923.68,4400230.86,4396643.54,4475996.61,4778560.19,4912357.20,5323877.68,...,6517107.95,6357582.34,6428430.50,7712972.01,8130900.15,7782972.05,7096426.57,6186918.31,5434024.79,2010-01-04
4,Sistema,Sistema,5044349.31,4810224.85,4711179.46,4644547.47,4821049.57,5066868.38,5233620.74,5612777.12,...,6648597.90,6586555.45,6617438.28,7888852.40,8206986.44,7800736.46,7115549.68,6250156.98,5605618.05,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,Sistema,Sistema,8518210.80,8198137.14,7966493.50,7848538.15,7993954.30,8373795.23,8533971.50,8989609.97,...,10224371.17,10138968.61,9944771.52,10267157.48,10388113.98,10186516.32,9774795.94,9269233.15,8783850.22,2024-04-26
5230,Sistema,Sistema,8416268.81,8139684.03,7934365.14,7801216.87,7802711.66,7866953.71,8060693.66,8611512.50,...,9613448.98,9515583.90,9453804.19,10022279.38,10070282.41,9806988.02,9391935.06,8934686.71,8515651.18,2024-04-27
5231,Sistema,Sistema,8153358.39,7823646.09,7495560.68,7290789.06,7186795.65,7058942.05,6899473.59,7167562.93,...,8373980.81,8391670.64,8366776.05,9037799.05,9296771.60,9195967.42,8965875.21,8547088.54,8101750.18,2024-04-28
5232,Sistema,Sistema,7755006.39,7559477.81,7424995.07,7358043.41,7541190.63,7993483.44,8218854.74,8737461.99,...,10365625.70,10315355.57,10040895.25,10521997.09,10741050.68,10522650.66,10127382.98,9476883.76,8842359.25,2024-04-29


In [132]:
df_generacion_ideal_sistema = df_generacion_ideal_sistema.set_index('Date')
df_generacion_ideal_sistema['generacion_ideal_sistema_daily'] = df_generacion_ideal_sistema.loc[:, 'Values_Hour01':'Values_Hour24'].sum(axis=1)
df_generacion_ideal_sistema_daily = df_generacion_ideal_sistema[['generacion_ideal_sistema_daily']]
df_generacion_ideal_sistema_daily


,generacion_ideal_sistema_daily
Date,
2010-01-01,1.154409e+08
2010-01-02,1.276983e+08
2010-01-03,1.236548e+08
2010-01-04,1.444114e+08
2010-01-05,1.490348e+08
...,...
2024-04-26,2.246048e+08
2024-04-27,2.171655e+08
2024-04-28,1.941931e+08


#### Union Dataset

In [133]:
df_consolidated = pd.merge(df_generacion_sistema_daily, df_generacion_ideal_sistema_daily, on="Date", how='outer')
df_consolidated

,generacion_sistema_daily,generacion_ideal_sistema_daily
Date,,
2010-01-01,1.154191e+08,1.154409e+08
2010-01-02,1.276812e+08,1.276983e+08
2010-01-03,1.236378e+08,1.236548e+08
2010-01-04,1.459575e+08,1.444114e+08
2010-01-05,1.521480e+08,1.490348e+08
...,...,...
2024-04-26,2.246048e+08,2.246048e+08
2024-04-27,2.171655e+08,2.171655e+08
2024-04-28,1.941888e+08,1.941931e+08


### Perdidas del Sistema
-   Metrica: Perdidas del sistema
-   Unidad: kWh	
-   Descripción: Cantidad de las inyecciones de energia al Sistema de Transmision Nacional (flujos de baja a alta tension), menos la cantidad de los flujos que salieron del Sistema (flujos de alta a baja tension).

In [134]:
sql_query = """SELECT * FROM perdidas_sistema"""
df_perdidas_sistema = pd.read_sql_query(sql_query, con=conn)
df_perdidas_sistema['Date'] = pd.to_datetime(df_perdidas_sistema['Date'])
df_perdidas_sistema

,Column1,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,0,Sistema,Sistema,105774.61,102118.61,83645.43,76479.29,86936.10,90440.22,63824.36,...,91227.73,72140.80,81412.93,101288.44,117722.66,105167.30,117369.59,117319.88,63782.69,2010-01-01
1,1,Sistema,Sistema,67249.86,68144.93,80334.54,71838.53,82755.38,73647.53,57781.19,...,76224.22,73456.17,80452.90,82028.96,123472.06,135808.67,136890.05,153906.03,138367.05,2010-01-02
2,2,Sistema,Sistema,85150.73,82398.64,72624.72,68972.34,94164.66,91730.56,63489.63,...,103027.98,99768.55,87914.68,91854.79,156818.60,148016.20,149224.20,147846.70,148218.55,2010-01-03
3,3,Sistema,Sistema,105862.54,105914.33,78999.12,73351.31,46846.53,69821.14,58761.25,...,114344.91,69098.95,63711.63,74057.35,128479.40,145726.62,146592.45,114760.41,41776.99,2010-01-04
4,4,Sistema,Sistema,76579.31,95728.29,86616.09,63866.76,79934.42,66618.63,59586.27,...,87785.94,82425.43,58046.85,128688.74,122294.09,110237.18,112819.66,135709.39,216910.06,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,5229,Sistema,Sistema,126262.35,125898.61,125845.56,116964.45,100126.47,112034.68,125497.24,...,169304.69,188228.31,245061.69,143956.22,152486.44,158102.02,154361.52,141111.40,145943.10,2024-04-26
5230,5230,Sistema,Sistema,145872.60,142324.34,140287.40,134343.81,128968.31,130514.03,118078.49,...,137922.40,146466.64,146785.72,150258.12,166424.90,164811.00,156758.48,138097.20,126882.77,2024-04-27
5231,5231,Sistema,Sistema,131308.39,125172.03,125330.72,126857.51,124666.49,118256.46,106743.08,...,126654.41,117944.98,108010.41,122707.43,129748.61,129918.48,129930.01,117732.00,110637.54,2024-04-28
5232,5232,Sistema,Sistema,97878.18,96572.57,95367.40,95241.84,78463.46,95768.18,101123.09,...,128655.82,142992.38,127562.01,136332.38,151891.98,148351.95,154116.17,133460.55,113115.72,2024-04-29


In [135]:
df_perdidas_sistema = df_perdidas_sistema.set_index('Date')
df_perdidas_sistema['perdidas_sistema_daily'] = df_perdidas_sistema.loc[:, 'Values_Hour01':'Values_Hour24'].sum(axis=1)
df_perdidas_sistema_daily = df_perdidas_sistema[['perdidas_sistema_daily']]
df_perdidas_sistema_daily

,perdidas_sistema_daily
Date,
2010-01-01,2079352.97
2010-01-02,2226467.46
2010-01-03,2509638.35
2010-01-04,2032914.64
2010-01-05,2278901.51
...,...
2024-04-26,3468541.50
2024-04-27,3317765.92
2024-04-28,2880163.08


In [136]:
df_consolidated = df_consolidated.join(df_perdidas_sistema_daily, on="Date")
df_consolidated

,generacion_sistema_daily,generacion_ideal_sistema_daily,perdidas_sistema_daily
Date,,,
2010-01-01,1.154191e+08,1.154409e+08,2079352.97
2010-01-02,1.276812e+08,1.276983e+08,2226467.46
2010-01-03,1.236378e+08,1.236548e+08,2509638.35
2010-01-04,1.459575e+08,1.444114e+08,2032914.64
2010-01-05,1.521480e+08,1.490348e+08,2278901.51
...,...,...,...
2024-04-26,2.246048e+08,2.246048e+08,3468541.50
2024-04-27,2.171655e+08,2.171655e+08,3317765.92
2024-04-28,1.941888e+08,1.941931e+08,2880163.08
